In [1]:
from glob import glob

In [2]:
files = glob("../data/hist_prices*")
print(files)

['../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_30.csv', '../data/hist_prices_2024-01-01-00-00_None_900_1000000000_(0, 0.4)_30.csv', '../data/hist_prices_2024-01-01-00-00_None_3600_1000000000_(0, 0.4)_30.csv', '../data/hist_prices_2024-01-01-00-00_None_900_10000000000_(0, 0.4)_45.csv', '../data/hist_prices_2024-01-01-00-00_None_60_10000000000_(0, 0.4)_30.csv', '../data/hist_prices_2024-01-01-00-00_None_300_1000000000_(0, 0.4)_30.csv', '../data/hist_prices_2024-01-01-00-00_None_86400_10000000000_(0, 0.4)_45.csv', '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_45.csv', '../data/hist_prices_2024-01-01-00-00_None_60_10000000000_(0, 0.4)_45.csv']


In [3]:
import pandas as pd

In [4]:
dfs = []
for file in files:
    dfs+=[pd.read_csv(file)]

In [5]:
dfs

[           time  UNI-USD  ALGO-USD  TIA-USD  LTC-USD  STX-USD  APT-USDT  \
 0    2024-01-01    7.605    0.2390   13.998    74.75   1.6246      9.87   
 1    2024-01-02    7.180    0.2269   12.783    72.98   1.5470     10.24   
 2    2024-01-03    6.436    0.2046   12.192    65.29   1.5548      9.08   
 3    2024-01-04    6.537    0.2075   15.630    66.49   1.7197     10.24   
 4    2024-01-05    6.436    0.1969   15.622    66.08   1.6112      9.66   
 ..          ...      ...       ...      ...      ...      ...       ...   
 119  2024-04-29    7.689    0.1901    9.744    83.48   2.4739      9.09   
 120  2024-04-30    7.033    0.1780    9.467    79.44   2.1564      8.76   
 121  2024-05-01    7.035    0.1805    9.679    80.05   2.1396      8.91   
 122  2024-05-02    7.107    0.1811   10.143    80.04   2.0854      8.87   
 123  2024-05-03    7.473    0.1908   10.171    81.94   2.1844      9.15   
 
      SAND-USDT  RNDR-USDT  ADA-USDT  ...  ARB-USD  AXS-USD  MATIC-USDT  \
 0        0

In [6]:
len(dfs)

9

In [7]:
pd.merge(dfs[0],dfs[1])

,time,UNI-USD,ALGO-USD,TIA-USD,LTC-USD,STX-USD,APT-USDT,SAND-USDT,RNDR-USDT,ADA-USDT,...,ETH-USDT,VET-USD,GRT-USD,CHZ-USD,ETC-USD,LDO-USD,CRO-USD,ADA-USD,STX-USDT,AVAX-USD


# classify files by granularity
- group files by granularity, each granularity need to be treated differently

In [13]:
granularities = {}
for file in files:
    file_params = file.split('/')[2].split('.[a-z]*')[0].split('_')[2:]
    if int(file_params[2]) not in granularities:
        granularities[int(file_params[2])] = []
    granularities[int(file_params[2])] += [{'file': file, 'start_date': file_params[0], 'end_date': file_params[1], 
    'market_cap': int(file_params[3]), 'bound': file_params[4], 'return_period': file_params[5]}]
granularities

{86400: [{'file': '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_30.csv',
   'start_date': '2024-01-01-00-00',
   'end_date': 'None',
   'market_cap': 1000000000,
   'bound': '(0, 0.4)',
   'return_period': '30.csv'},
  {'file': '../data/hist_prices_2024-01-01-00-00_None_86400_10000000000_(0, 0.4)_45.csv',
   'start_date': '2024-01-01-00-00',
   'end_date': 'None',
   'market_cap': 10000000000,
   'bound': '(0, 0.4)',
   'return_period': '45.csv'},
  {'file': '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_45.csv',
   'start_date': '2024-01-01-00-00',
   'end_date': 'None',
   'market_cap': 1000000000,
   'bound': '(0, 0.4)',
   'return_period': '45.csv'}],
 900: [{'file': '../data/hist_prices_2024-01-01-00-00_None_900_1000000000_(0, 0.4)_30.csv',
   'start_date': '2024-01-01-00-00',
   'end_date': 'None',
   'market_cap': 1000000000,
   'bound': '(0, 0.4)',
   'return_period': '30.csv'},
  {'file': '../data/hist_prices_2024-01-01-00-00_None_9

# merge, union assumptions

- price history is indifferent to bound.
- price history is indifferent to return_period.
- price history depends on granularity.
- price history is filtered by market_cap
- 


In [14]:
def filtered_granularity(granularity, market_cap):
    gran_filtered = granularities[granularity]
    mc_filtered = [i for i in gran_filtered if i['market_cap'] >= market_cap]
    return mc_filtered

    

In [15]:
gran_mc_filtered_files = filtered_granularity(86400, 10**9)
gran_mc_filtered_files

[{'file': '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_30.csv',
  'start_date': '2024-01-01-00-00',
  'end_date': 'None',
  'market_cap': 1000000000,
  'bound': '(0, 0.4)',
  'return_period': '30.csv'},
 {'file': '../data/hist_prices_2024-01-01-00-00_None_86400_10000000000_(0, 0.4)_45.csv',
  'start_date': '2024-01-01-00-00',
  'end_date': 'None',
  'market_cap': 10000000000,
  'bound': '(0, 0.4)',
  'return_period': '45.csv'},
 {'file': '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_45.csv',
  'start_date': '2024-01-01-00-00',
  'end_date': 'None',
  'market_cap': 1000000000,
  'bound': '(0, 0.4)',
  'return_period': '45.csv'}]

In [16]:
from datetime import datetime

In [17]:
def adjust_dates(gran_mc_filtered_files, timefmt='%Y-%m-%d'):
    adjusted_dates_files = []
    for file in gran_mc_filtered_files:
        df = pd.read_csv(file['file'])
        adjusted_start_date = datetime.strptime(df.iloc[0]['time'], timefmt)
        adjusted_end_date = datetime.strptime(df.iloc[-1]['time'], timefmt)
        adjusted_dates_files += [{'file':file['file'], 'start_date':adjusted_start_date, 'end_date': adjusted_end_date}]
    return adjusted_dates_files

In [18]:
date_adjusted_files = adjust_dates(gran_mc_filtered_files)
date_adjusted_files

[{'file': '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_30.csv',
  'start_date': datetime.datetime(2024, 1, 1, 0, 0),
  'end_date': datetime.datetime(2024, 5, 3, 0, 0)},
 {'file': '../data/hist_prices_2024-01-01-00-00_None_86400_10000000000_(0, 0.4)_45.csv',
  'start_date': datetime.datetime(2024, 1, 1, 0, 0),
  'end_date': datetime.datetime(2024, 5, 5, 0, 0)},
 {'file': '../data/hist_prices_2024-01-01-00-00_None_86400_1000000000_(0, 0.4)_45.csv',
  'start_date': datetime.datetime(2024, 1, 1, 0, 0),
  'end_date': datetime.datetime(2024, 5, 3, 0, 0)}]

In [19]:
dfs = []
for file in date_adjusted_files:
    dfs += [pd.read_csv(file['file'])]
dfs  

[           time  UNI-USD  ALGO-USD  TIA-USD  LTC-USD  STX-USD  APT-USDT  \
 0    2024-01-01    7.605    0.2390   13.998    74.75   1.6246      9.87   
 1    2024-01-02    7.180    0.2269   12.783    72.98   1.5470     10.24   
 2    2024-01-03    6.436    0.2046   12.192    65.29   1.5548      9.08   
 3    2024-01-04    6.537    0.2075   15.630    66.49   1.7197     10.24   
 4    2024-01-05    6.436    0.1969   15.622    66.08   1.6112      9.66   
 ..          ...      ...       ...      ...      ...      ...       ...   
 119  2024-04-29    7.689    0.1901    9.744    83.48   2.4739      9.09   
 120  2024-04-30    7.033    0.1780    9.467    79.44   2.1564      8.76   
 121  2024-05-01    7.035    0.1805    9.679    80.05   2.1396      8.91   
 122  2024-05-02    7.107    0.1811   10.143    80.04   2.0854      8.87   
 123  2024-05-03    7.473    0.1908   10.171    81.94   2.1844      9.15   
 
      SAND-USDT  RNDR-USDT  ADA-USDT  ...  ARB-USD  AXS-USD  MATIC-USDT  \
 0        0

In [20]:
df = pd.concat(dfs)
df=df.set_index('time')
df

,UNI-USD,ALGO-USD,TIA-USD,LTC-USD,STX-USD,APT-USDT,SAND-USDT,RNDR-USDT,ADA-USDT,WBTC-USD,...,CRO-USD,SAND-USD,ETC-USD,MATIC-USD,RNDR-USD,HBAR-USD,LDO-USD,ICP-USDT,CHZ-USDT,AXS-USDT
time,,,,,,,,,,,,,,,,,,,,,
2024-01-01,7.605,0.2390,13.998,74.75,1.6246,9.87,0.602,4.7333,0.623,44192.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-02,7.180,0.2269,12.783,72.98,1.5470,10.24,0.587,4.6611,0.603,44957.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-03,6.436,0.2046,12.192,65.29,1.5548,9.08,0.515,4.4171,0.558,42989.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,6.537,0.2075,15.630,66.49,1.7197,10.24,0.539,4.3820,0.569,44294.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-05,6.436,0.1969,15.622,66.08,1.6112,9.66,0.511,4.0120,0.542,44156.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-29,7.689,0.1901,9.744,83.48,2.4739,9.09,NaN,NaN,NaN,63763.21,...,0.1319,0.4433,27.58,0.7039,7.8770,0.10324,2.193,13.39,0.1140,7.21
2024-04-30,7.033,0.1780,9.467,79.44,2.1564,8.76,NaN,NaN,NaN,60710.40,...,0.1347,0.4179,25.40,0.6669,7.3389,0.09371,1.870,12.89,0.1086,6.72
2024-05-01,7.035,0.1805,9.679,80.05,2.1396,8.91,NaN,NaN,NaN,58222.00,...,0.1273,0.4242,25.51,0.6912,7.6578,0.09949,1.937,13.17,0.1165,7.00


In [21]:
list(df.columns)

['UNI-USD',
 'ALGO-USD',
 'TIA-USD',
 'LTC-USD',
 'STX-USD',
 'APT-USDT',
 'SAND-USDT',
 'RNDR-USDT',
 'ADA-USDT',
 'WBTC-USD',
 'ATOM-USD',
 'CRO-USDT',
 'MKR-USD',
 'DOGE-USDT',
 'XLM-USD',
 'BCH-USD',
 'XRP-USD',
 'IMX-USD',
 'FLR-USD',
 'INJ-USD',
 'SHIB-USD',
 'AVAX-USDT',
 'SOL-USDT',
 'ICP-USD',
 'DOT-USDT',
 'SUI-USD',
 'LINK-USD',
 'BIT-USD',
 'FLOW-USD',
 'QNT-USD',
 'NEAR-USD',
 'FIL-USD',
 'AKT-USD',
 'HBAR-USDT',
 'ARB-USD',
 'AXS-USD',
 'MATIC-USDT',
 'AAVE-USD',
 'SEI-USD',
 'BTC-USDT',
 'ETH-USDT',
 'VET-USD',
 'GRT-USD',
 'CHZ-USD',
 'ADA-USD',
 'ETH-USD',
 'DOT-USD',
 'DOGE-USD',
 'AVAX-USD',
 'SOL-USD',
 'CRO-USD',
 'SAND-USD',
 'ETC-USD',
 'MATIC-USD',
 'RNDR-USD',
 'HBAR-USD',
 'LDO-USD',
 'ICP-USDT',
 'CHZ-USDT',
 'AXS-USDT']

In [22]:
import numpy as np

In [23]:
gdf = df.groupby('time')
gdf

In [24]:
gdf.first()

,UNI-USD,ALGO-USD,TIA-USD,LTC-USD,STX-USD,APT-USDT,SAND-USDT,RNDR-USDT,ADA-USDT,WBTC-USD,...,CRO-USD,SAND-USD,ETC-USD,MATIC-USD,RNDR-USD,HBAR-USD,LDO-USD,ICP-USDT,CHZ-USDT,AXS-USDT
time,,,,,,,,,,,,,,,,,,,,,
2024-01-01,7.605,0.2390,13.998,74.75,1.6246,9.87,0.602,4.7333,0.623,44192.15,...,0.1008,0.6042,22.50,1.0161,4.7438,0.09157,2.915,12.93,0.0894,9.22
2024-01-02,7.180,0.2269,12.783,72.98,1.5470,10.24,0.587,4.6611,0.603,44957.00,...,0.1038,0.5872,21.98,0.9689,4.6605,0.09706,2.866,14.03,0.0876,8.96
2024-01-03,6.436,0.2046,12.192,65.29,1.5548,9.08,0.515,4.4171,0.558,42989.40,...,0.0960,0.5169,20.04,0.8544,4.4185,0.08848,3.068,14.37,0.0807,7.94
2024-01-04,6.537,0.2075,15.630,66.49,1.7197,10.24,0.539,4.3820,0.569,44294.87,...,0.0976,0.5389,20.50,0.8840,4.3783,0.08802,3.125,13.87,0.0821,8.40
2024-01-05,6.436,0.1969,15.622,66.08,1.6112,9.66,0.511,4.0120,0.542,44156.04,...,0.0922,0.5125,20.15,0.8538,4.0133,0.08489,3.565,13.18,0.0795,7.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,7.035,0.1805,9.679,80.05,2.1396,8.91,0.424,7.6596,0.450,58222.00,...,0.1273,0.4242,25.51,0.6912,7.6578,0.09949,1.937,13.17,0.1165,7.00
2024-05-02,7.107,0.1811,10.143,80.04,2.0854,8.87,0.435,7.8293,0.460,59151.13,...,0.1279,0.4342,25.65,0.7206,7.8254,0.10033,1.976,13.60,0.1185,7.37
2024-05-03,7.473,0.1908,10.171,81.94,2.1844,9.15,0.451,8.1033,0.468,61787.20,...,0.1331,0.4486,26.58,0.7310,8.0601,0.10850,2.161,13.57,0.1188,7.54


In [25]:
adf = pd.DataFrame()
for col in df.columns:
    adf[col] = gdf[col].agg(np.sum)


/tmp/ipykernel_239322/3092838503.py:3: FutureWarning: The provided callable <function sum at 0x7fdc5c1b35b0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  adf[col] = gdf[col].agg(np.sum)
/tmp/ipykernel_239322/3092838503.py:3: FutureWarning: The provided callable <function sum at 0x7fdc5c1b35b0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  adf[col] = gdf[col].agg(np.sum)
/tmp/ipykernel_239322/3092838503.py:3: FutureWarning: The provided callable <function sum at 0x7fdc5c1b35b0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  adf[col] = gdf[col].agg(np.sum)
/tmp/ipykernel_239322/3092838503.py:3: FutureWarning: The pro

In [26]:
adf

,UNI-USD,ALGO-USD,TIA-USD,LTC-USD,STX-USD,APT-USDT,SAND-USDT,RNDR-USDT,ADA-USDT,WBTC-USD,...,CRO-USD,SAND-USD,ETC-USD,MATIC-USD,RNDR-USD,HBAR-USD,LDO-USD,ICP-USDT,CHZ-USDT,AXS-USDT
time,,,,,,,,,,,,,,,,,,,,,
2024-01-01,15.210,0.4780,27.996,149.50,3.2492,19.74,0.602,4.7333,0.623,88384.30,...,0.1008,0.6042,22.50,1.0161,4.7438,0.09157,2.915,12.93,0.0894,9.22
2024-01-02,14.360,0.4538,25.566,145.96,3.0940,20.48,0.587,4.6611,0.603,89914.00,...,0.1038,0.5872,21.98,0.9689,4.6605,0.09706,2.866,14.03,0.0876,8.96
2024-01-03,12.872,0.4092,24.384,130.58,3.1096,18.16,0.515,4.4171,0.558,85978.80,...,0.0960,0.5169,20.04,0.8544,4.4185,0.08848,3.068,14.37,0.0807,7.94
2024-01-04,13.074,0.4150,31.260,132.98,3.4394,20.48,0.539,4.3820,0.569,88589.74,...,0.0976,0.5389,20.50,0.8840,4.3783,0.08802,3.125,13.87,0.0821,8.40
2024-01-05,12.872,0.3938,31.244,132.16,3.2224,19.32,0.511,4.0120,0.542,88312.08,...,0.0922,0.5125,20.15,0.8538,4.0133,0.08489,3.565,13.18,0.0795,7.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,14.070,0.3610,19.358,160.10,4.2792,17.82,0.424,7.6596,0.450,116444.00,...,0.1273,0.4242,25.51,0.6912,7.6578,0.09949,1.937,13.17,0.1165,7.00
2024-05-02,14.214,0.3622,20.286,160.08,4.1708,17.74,0.435,7.8293,0.460,118302.26,...,0.1279,0.4342,25.65,0.7206,7.8254,0.10033,1.976,13.60,0.1185,7.37
2024-05-03,14.911,0.3823,20.159,163.85,4.3547,18.27,0.451,8.1033,0.468,123263.37,...,0.1331,0.4486,26.58,0.7310,8.0601,0.10850,2.161,13.57,0.1188,7.54


In [28]:
complete_adf = pd.DataFrame()

# concatenation assumptions and rules

- implement concatenation in a loop for simplicity
- NaN = 0 for price history
- after concatenation of two data frames, nan/zeros could only be at the begining, or the end of the price history
- zero values at the end of price history are missing values, and could be retrieved.
- missing values at beging of history could occure before token launch.

In [35]:
def get_token_initial_price_date(token):
    #TODO implement
    return '2022-01-01-00-00'

In [52]:
def get_nan_intervals(token_series):
    mask = token_series == 0
    start_date_idx = len(mask)-1
    end_date_idx = len(mask)-1
    intervals = []
    for i in range(len(mask)):
        # if nan is encountered
        if mask[i]:
            # if it's begining of nan interval
            if i==0 or mask[i-1]==False: 
                start_date_idx=i
            # if it's end of interval at end of series
            if i==len(mask)-1 and mask[i-1]:
                end_date_idx=i
                intervals+=[(token_series.index[start_date_idx], token_series.index[end_date_idx])]
        # if not NaN is encountered
        else:
            # if NaN range just ended, add interval, and set end_date_idx
            if i>0 and mask[i-1]:
                end_date_idx=i-1
                intervals+=[(token_series.index[start_date_idx], token_series.index[end_date_idx])]
    return intervals    

In [53]:
intervals = get_nan_intervals(adf['UNI-USD'])
intervals

/tmp/ipykernel_239322/796289051.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if mask[i]:
/tmp/ipykernel_239322/796289051.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i>0 and mask[i-1]:
/tmp/ipykernel_239322/796289051.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if i==0 or mask[i-1]==False:
/tmp/ipykernel_239322/796289051.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a

[('2024-05-04', '2024-05-05')]

only retrive first, and last interval, according to assumptions above

In [44]:
intervals = intervals if len(intervals)<3 else [intervals[0], intervals[-1]]

merge, and get tokens missing intervals (complete algorithm)

In [ ]:
def merge_dfs_missing(dfs):
    df = pd.DataFrame()
    merge_intervals = []
    for df_i in dfs:
        df = pd.concat([df, df_i])
        df=df.set_index('time')
        gdf = df.groupby('time')
        adf = pd.DataFrame()
        for col in df.columns:
            adf[col] = gdf[col].agg(np.mean)
        col_intervals = {}
        for col in adf.columns:
            intervals = get_nan_intervals(adf[col])
            intervals = intervals if len(intervals)<3 else [intervals[0], intervals[-1]]
            col_intervals[col] = intervals
        #TODO in order for this to work, you must retrive missing intervals,
        # before proceeding further.
    return adf, merge_intervals
        
        
        

    

In [ ]:
def merge_dfs_intervals(dfs):
    df = pd.DataFrame()
    df = pd.concat(dfs)
    df=df.set_index('time')
    gdf = df.groupby('time')
    adf = pd.DataFrame()
    for col in df.columns:
        adf[col] = gdf[col].agg(np.mean)
    col_intervals = {}
    for col in adf.columns:
        interval = get_nan_intervals(adf[col])
        col_intervals[col] = interval
    return adf, col_intervals